In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/merged-caltech-weather-traffic/merged_data_wt2.csv
/kaggle/input/merged-caltech-weather-traffic/merged_data_wt.csv
/kaggle/input/caltech1/caltech1.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/kaggle/input/merged-caltech-weather-traffic/merged_data_wt2.csv')


In [4]:
df.columns

Index(['Unnamed: 0', 'sessionID', 'stationID', 'spaceID', 'connectionTime',
       'disconnectTime', 'kWhDelivered', 'doneChargingTime', 'timezone',
       'userID', 'session_duration', 'Weekend', 'connectionDate', 'clouds',
       'datetime', 'dewpt', 'max_uv', 'max_wind_spd', 'min_temp', 'precip',
       'pres', 'revision_status', 'rh', 'slp', 'temp', 'wind_dir', 'wind_spd',
       'Date', 'TrafficVolume', 'AverageSpeed'],
      dtype='object')

In [5]:
df = df.drop(['Unnamed: 0','sessionID','connectionTime', 'spaceID', 'userID','disconnectTime','doneChargingTime','timezone','datetime','connectionDate', 'Date', 'revision_status','stationID'], axis=1)

In [6]:
df.shape

(1945, 17)

In [7]:
df.columns

Index(['kWhDelivered', 'session_duration', 'Weekend', 'clouds', 'dewpt',
       'max_uv', 'max_wind_spd', 'min_temp', 'precip', 'pres', 'rh', 'slp',
       'temp', 'wind_dir', 'wind_spd', 'TrafficVolume', 'AverageSpeed'],
      dtype='object')

In [8]:
df.head()

,kWhDelivered,session_duration,Weekend,clouds,dewpt,max_uv,max_wind_spd,min_temp,precip,pres,rh,slp,temp,wind_dir,wind_spd,TrafficVolume,AverageSpeed
0,39.894,1049.750000,0,51,1.6,2.9,3.4,1.5,0.8,987,80.0,1011,5.0,172,1.4,175.16875,34.242583
1,10.627,198.066667,0,51,1.6,2.9,3.4,1.5,0.8,987,80.0,1011,5.0,172,1.4,175.16875,34.242583
2,30.479,288.916667,1,92,1.6,1.5,2.4,2.4,1.5,997,73.5,1021,6.3,53,1.6,175.16875,34.242583
3,1.650,92.283333,1,92,1.6,1.5,2.4,2.4,1.5,997,73.5,1021,6.3,53,1.6,175.16875,34.242583
4,7.049,127.866667,1,92,1.6,1.5,2.4,2.4,1.5,997,73.5,1021,6.3,53,1.6,175.16875,34.242583


In [9]:
df.columns

Index(['kWhDelivered', 'session_duration', 'Weekend', 'clouds', 'dewpt',
       'max_uv', 'max_wind_spd', 'min_temp', 'precip', 'pres', 'rh', 'slp',
       'temp', 'wind_dir', 'wind_spd', 'TrafficVolume', 'AverageSpeed'],
      dtype='object')

In [10]:
data = df

In [11]:
# Identify categorical columns based on data types
data_types = data.dtypes
categorical_columns = data_types[data_types == 'object'].index.tolist()

# Check unique values in categorical columns to confirm if they are indeed categorical
for column in categorical_columns:
    unique_values = data[column].unique()
    print(f"Unique values in {column}: {unique_values}")

# You can also set a threshold to determine if a column is categorical based on the number of unique values
threshold_unique_values = 10  # Adjust threshold as needed
categorical_columns = [column for column in data.columns if len(data[column].unique()) <= threshold_unique_values]

# Print the identified categorical columns
print("Categorical Columns:", categorical_columns)

# Non-categorical columns
non_categorical_columns = [column for column in data.columns if column not in categorical_columns]
print("Non-Categorical Columns:", non_categorical_columns)

Categorical Columns: ['Weekend']
Non-Categorical Columns: ['kWhDelivered', 'session_duration', 'clouds', 'dewpt', 'max_uv', 'max_wind_spd', 'min_temp', 'precip', 'pres', 'rh', 'slp', 'temp', 'wind_dir', 'wind_spd', 'TrafficVolume', 'AverageSpeed']


In [12]:
# import pandas as pd

# # Assuming 'data' is your DataFrame
# # List of categorical columns
# categorical_columns = ['spaceID', 'userID']

# # Apply one-hot encoding for each categorical column
# for column in categorical_columns:
#     # Convert the column to one-hot encoded representation
#     one_hot_encoded = pd.get_dummies(data[column], prefix=column)
#     # Concatenate the one-hot encoded columns to the original dataframe
#     data = pd.concat([data, one_hot_encoded], axis=1)
#     # Drop the original categorical column from the dataframe
#     data.drop(column, axis=1, inplace=True)


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
   
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data)
data_imputed = pd.DataFrame(data_imputed, columns=data.columns)

data_imputed = data_imputed.drop(['session_duration'], axis=1)

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)


pca = PCA(n_components=15)  
data_pca = pca.fit_transform(data_scaled)



In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

# Assuming 'data' contains your dataset

# Impute missing values
imputer = SimpleImputer(strategy='mean')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Drop target variable before PCA
X = data_imputed.drop(['session_duration'], axis=1)

# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform PCA
pca = PCA(n_components=15)
X_pca = pca.fit_transform(X_scaled)

# Define target variable
y = data_imputed['session_duration']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2)

In [15]:
# #X = data.drop(columns=['session_duration'])
# X = data_pca
# y = data['session_duration']

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

# Machine Learning Model - Random Forest
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
rf_score = rf_model.score(X_test, y_test)

# Machine Learning Model - XGBoost
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
xgb_score = xgb_model.score(X_test, y_test)

# Machine Learning Model - LightGBM
lgbm_model = LGBMRegressor()
lgbm_model.fit(X_train, y_train)
lgbm_score = lgbm_model.score(X_test, y_test)

# Machine Learning Model - SVM
svm_model = SVR()
svm_model.fit(X_train, y_train)
svm_score = svm_model.score(X_test, y_test)


# Predictions
rf_predictions = rf_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
lgbm_predictions = lgbm_model.predict(X_test)
svm_predictions = svm_model.predict(X_test)


# Calculate RMSE and MAE for Random Forest
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
rf_mae = mean_absolute_error(y_test, rf_predictions)

# Calculate RMSE and MAE for XGBoost
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_predictions))
xgb_mae = mean_absolute_error(y_test, xgb_predictions)

# Calculate RMSE and MAE for LightGBM
lgbm_rmse = np.sqrt(mean_squared_error(y_test, lgbm_predictions))
lgbm_mae = mean_absolute_error(y_test, lgbm_predictions)

# Calculate RMSE and MAE for SVM
svm_rmse = np.sqrt(mean_squared_error(y_test, svm_predictions))
svm_mae = mean_absolute_error(y_test, svm_predictions)



# Print the evaluation metrics
print("Random Forest Metrics:")
print("RMSE:", rf_rmse)
print("MAE:", rf_mae)
print("\nXGBoost Metrics:")
print("RMSE:", xgb_rmse)
print("MAE:", xgb_mae)
print("\nLightGBM Metrics:")
print("RMSE:", lgbm_rmse)
print("MAE:", lgbm_mae)
print("\nSVM Metrics:")
print("RMSE:", svm_rmse)
print("MAE:", svm_mae)



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001906 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3825
[LightGBM] [Info] Number of data points in the train set: 1556, number of used features: 15
[LightGBM] [Info] Start training from score 336.777024
Random Forest Metrics:
RMSE: 237.27898342857137
MAE: 174.07185304198802

XGBoost Metrics:
RMSE: 244.29276983257319
MAE: 175.75918199768816

LightGBM Metrics:
RMSE: 223.69974249618048
MAE: 163.9009217451401

SVM Metrics:
RMSE: 225.58836737902635
MAE: 178.91893198169618


In [17]:
# # Feature Selection
# model = RandomForestRegressor()
# model.fit(data_pca, data_imputed['session_duration'])  # Replace 'target_column' with your target variable
# feature_importances = model.feature_importances_
# selector = SelectFromModel(model, prefit=True)
# selected_features = selector.transform(data_imputeda)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(selected_features, data_imputed['session_duration'], test_size=0.2)

# # Machine Learning Model - Random Forest
# rf_model = RandomForestRegressor()
# rf_model.fit(X_train, y_train)
# rf_score = rf_model.score(X_test, y_test)

# # Machine Learning Model - XGBoost
# xgb_model = XGBRegressor()
# xgb_model.fit(X_train, y_train)
# xgb_score = xgb_model.score(X_test, y_test)

# # print("Random Forest Score:", rf_score)
# # print("XGBoost Score:", xgb_score)
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# import numpy as np

# # Predictions
# rf_predictions = rf_model.predict(X_test)
# xgb_predictions = xgb_model.predict(X_test)

# # Calculate RMSE and MAE for Random Forest
# rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
# rf_mae = mean_absolute_error(y_test, rf_predictions)

# # Calculate RMSE and MAE for XGBoost
# xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_predictions))
# xgb_mae = mean_absolute_error(y_test, xgb_predictions)

# # Print the evaluation metrics
# print("Random Forest Metrics:")
# print("RMSE:", rf_rmse)
# print("MAE:", rf_mae)
# print("\nXGBoost Metrics:")
# print("RMSE:", xgb_rmse)
# print("MAE:", xgb_mae)

